### Importación de librerías

In [268]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random
import sys
import pickle
import os
import pygame
import ctypes
import threading
import pyttsx3
from PIL import ImageFont, ImageDraw, Image 

### Configuración y rutas

In [269]:
FILE_MANOS = "../train/modelo_body_red_augmentation.pkl"
FILE_CARA = "../train/modelo_facial_red_augmentation.pkl"

# Rutas de recursos (Fuentes y Sonidos)
BASE_PATH = os.getcwd()
FONT_CUSTOM = os.path.join(BASE_PATH, "fonts", "HelloMissDi.otf")
SOUND_SUCCESS = os.path.join(BASE_PATH, "sounds", "success.mp3") 
SOUND_FAIL = os.path.join(BASE_PATH, "sounds", "error.mp3")

# --- COLORES & ESTILOS (Paleta Neon/Arcade) ---
COLORS = {
    'neon_yellow': (0, 200, 255),  
    'neon_green':  (50, 255, 50),   
    'neon_purple': (255, 0, 255),   
    'neon_blue':   (255, 255, 0),   
    'neon_orange': (0, 165, 255),   
    'ligth_grey':  (255, 200, 200), 
    'white':       (255, 255, 255),
    'black':       (0, 0, 0)
}

COLORES_ANIMACION = [
    COLORS['neon_yellow'],
    COLORS['neon_green'],
    COLORS['neon_blue'],
    COLORS['neon_orange'],
    COLORS['neon_purple'],
]

# --- 1. Texto para mostrar qué debe hacer el usuario (INSTRUCCIÓN) ---
INSTRUCCIONES = {
    "1_Mano_Der_Arriba": "Levanta la mano derecha",
    "2_Mano_Izq_Arriba": "Levanta la mano izquierda",
    "3_Punos_Cerrados": "Cierra tu puño",
    "4_Pulgar_Arriba": "Pulgar hacia arriba",
    "5_Victoria": "Haz el símbolo de Victoria o Paz",
    "6_Rock": "Haz los cuernos del Rock",
    "7_Llamada": "Haz el gesto de llamar",
    "8_Ok": "Haz el gesto de Ok",
    "1_Ojos_Cerrados": "Cierra los ojos",
    "2_Cabeza_Der": "Mueve la cabeza hacia la derecha",
    "3_Cabeza_Izq": "Mueve la cabeza hacia la izquierda",
    "0_Neutro": "Neutro"
}

# --- 2. Texto para mostrar qué está detectando el sistema (FEEDBACK) ---
DETECCION = {
    "1_Mano_Der_Arriba": "Mano Arriba",
    "2_Mano_Izq_Arriba": "Mano Arriba",
    "3_Punos_Cerrados": "Puño",
    "4_Pulgar_Arriba": "Pulgar Arriba",
    "5_Victoria": "Victoria",
    "6_Rock": "Rock",
    "7_Llamada": "Llamada",
    "8_Ok": "OK",
    "1_Ojos_Cerrados": "Ojos Cerrados",
    "2_Cabeza_Der": "Cabeza Derecha",
    "3_Cabeza_Izq": "Cabeza Izquierda",
    "0_Neutro": "Neutro"
}

# --- 3. Texto para gramática de error "Tenías que..." (ERROR) ---
ERROR = {
    "1_Mano_Der_Arriba": "levantar la mano derecha",
    "2_Mano_Izq_Arriba": "levantar la mano izquierda",
    "3_Punos_Cerrados": "cerrar el puño",
    "4_Pulgar_Arriba": "levantar el pulgar",
    "5_Victoria": "hacer el símbolo de la paz",
    "6_Rock": "hacer el gesto de Rock",
    "7_Llamada": "hacer el gesto de llamada",
    "8_Ok": "hacer el gesto de Ok",
    "1_Ojos_Cerrados": "cerrar los ojos",
    "2_Cabeza_Der": "mover la cabeza hacia la derecha",
    "3_Cabeza_Izq": "mover la cabeza hacia la izquierda",
    "0_Neutro": "quedarte neutro"
}

### Procesamiento de Landmarks
Funciones para normalizar las coordenadas de los landmarks de manos y rostro, haciéndolos invariantes a la posición en la pantalla.

In [270]:
def get_hand_features(landmarks):
    """
    Extrae y normaliza las características de la mano.
    Centra las coordenadas en el promedio (centroide) y escala
    para que la detección sea invariante a la posición y distancia de la cámara.
    """
    coords = np.array([[lm.x, lm.y] for lm in landmarks.landmark])
    centroid = np.mean(coords, axis=0)
    centered = coords - centroid
    max_dist = np.max(np.abs(centered))
    if max_dist > 0:
        normalized = centered / max_dist
    else:
        normalized = centered
    return normalized.flatten()

def get_face_features(landmarks):
    """
    Extrae y normaliza las características faciales.
    Sigue el mismo proceso de centrado y escalado que la función de manos.
    """
    coords = np.array([[lm.x, lm.y] for lm in landmarks.landmark])
    centroid = np.mean(coords, axis=0)
    centered = coords - centroid
    max_dist = np.max(np.abs(centered))
    if max_dist > 0:
        normalized = centered / max_dist
    else:
        normalized = centered
    return normalized.flatten()

### Clase principal del juego
Contiene la lógica de `SimonGame`, incluyendo carga de modelos, detección de gestos, control de estados y renderizado de la interfaz (UI).

In [271]:

class SimonGame:
    def __init__(self):
        """
        Inicializa la instancia del juego, cargando modelos, 
        configurando MediaPipe y estableciendo variables de estado iniciales.
        """
        print("--- SIMON SAYS: MEMORY ---")
        
        # --- Carga de Modelos de Machine Learning ---
        if not os.path.exists(FILE_MANOS) or not os.path.exists(FILE_CARA):
            raise FileNotFoundError("Faltan modelos .pkl")

        with open(FILE_MANOS, 'rb') as f: self.model_hand = pickle.load(f)
        with open(FILE_CARA, 'rb') as f: self.model_face = pickle.load(f)

        # Mapeos internos de las predicciones numéricas a etiquetas de texto
        self.map_hand = {
            0: "0_Neutro", 1: "1_Mano_Der_Arriba", 2: "2_Mano_Izq_Arriba", 3: "3_Punos_Cerrados",
            4: "4_Pulgar_Arriba", 5: "5_Victoria", 6: "6_Rock", 7: "7_Llamada", 8: "8_Ok"
        }
        self.map_face = {
            0: "0_Neutro", 1: "1_Ojos_Cerrados", 2: "2_Cabeza_Der", 3: "3_Cabeza_Izq"
        }

        # --- Inicialización de Audio (Pygame) ---
        try:
            pygame.mixer.init()
            # Cargamos los sonidos. Si no existen, no crashea, solo avisa.
            if os.path.exists(SOUND_SUCCESS):
                self.sfx_success = pygame.mixer.Sound(SOUND_SUCCESS)
                self.sfx_success.set_volume(0.5) # Volumen al 50%
            else:
                print(f"Aviso: No se encontró sonido en {SOUND_SUCCESS}")
                self.sfx_success = None
            
            # Cargar sonido de error
            if os.path.exists(SOUND_FAIL):
                self.sfx_fail = pygame.mixer.Sound(SOUND_FAIL)
                self.sfx_fail.set_volume(0.4)
            else:
                self.sfx_fail = None
                
        except: pass

        # --- VOZ (Pyttsx3) ---
        # Inicializar el motor de voz
        self.engine = pyttsx3.init()

        voices = self.engine.getProperty('voices')
        for voice in voices:
            if "spanish" in voice.name.lower() or "es-es" in voice.id.lower():
                self.engine.setProperty('voice', voice.id)
                break
        
        self.engine.setProperty('rate', 145)

        # --- Configuración de MediaPipe ---
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5)
        self.mp_face = mp.solutions.face_mesh
        self.face_mesh = self.mp_face.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)
        self.mp_draw = mp.solutions.drawing_utils

        # Generación de lista de acciones disponibles filtrando "Neutro"
        self.acciones_disponibles = []
        all_labels = list(self.map_hand.values()) + list(self.map_face.values())
        for label in all_labels:
            if "Neutro" not in label and label not in self.acciones_disponibles:
                self.acciones_disponibles.append(label)

        # --- Variables de Control del Juego ---
        self.cap = cv2.VideoCapture(0)
        
        # --- MODIFICACIÓN: Pantalla completa / Resolución Pantalla ---
        try:
            # Obtener resolución de pantalla en Windows
            user32 = ctypes.windll.user32
            self.screen_w = user32.GetSystemMetrics(0)
            self.screen_h = user32.GetSystemMetrics(1)
            
            # Intentar configurar la cámara a esa resolución (o la máxima posible)
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.screen_w)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.screen_h)
            
            # Configurar ventana a Fullscreen 
        except Exception as e:
            print(f"Error ajustando resolución: {e}")

        self.state = "MENU"         # Estado inicial
        self.sequence = []          # Secuencia de pasos acumulada
        self.current_step_idx = 0   # Índice actual dentro de la secuencia
        self.score = 0
        self.timer_start = 0
        
        # --- Configuración de Tiempos (Dificultad) ---
        self.tiempo_memorizar = 2.0     # Segundos para ver el nuevo paso
        self.turn_time = 8.0            # Segundos para ejecutar la acción
        self.tiempo_entre_rondas = 4.0  # Pausa tras éxito
        self.tiempo_espera_trampa = 4.0
        
        # Variables para detección estable
        self.fail_reason = ""
        self.last_gesture = "0_Neutro"
        self.consecutive_frames = 0
        self.waiting_neutral = False 
        self.neutral_start_time = None

        # Flag para controlar que la voz hable solo una vez por paso
        self.speech_triggered = False

    def play_sound(self, sound_type):
        """Reproduce un sonido basado en el tipo de evento."""
        if sound_type == "success" and self.sfx_success:
            self.sfx_success.play()
        elif sound_type == "fail" and self.sfx_fail:
            self.sfx_fail.play()

    def hablar(self, texto):
        """Ejecuta la voz en un hilo separado para no congelar la cámara."""
        def _speak():
            try:
                if self.engine._inLoop: self.engine.endLoop()
                self.engine.say(texto)
                self.engine.runAndWait()
            except: pass
        thread = threading.Thread(target=_speak)
        thread.start()

    def poner_texto_utf8(self, frame, text, xy, color=(255,255,255), size=30, font_path=None):
        """
        Dibuja texto sobre el frame utilizando la librería Pillow.
        Permite el uso de caracteres UTF-8 (tildes, ñ) y fuentes personalizadas.
        Convierte de BGR (OpenCV) a RGB (PIL) y viceversa.
        """
        # 1. Convertir imagen de OpenCV (BGR) a Pillow (RGB)
        img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(img_pil)

        # 2. Convertir el color recibido (BGR) a RGB para que Pillow lo entienda
        color_rgb = color[::-1]
        
        font = None
        
        # 3. Intento de cargar la fuente personalizada
        if font_path is not None:
            try:
                font = ImageFont.truetype(font_path, size)
            except: pass

        # 4. Intento de cargar Arial del sistema Windows
        if font is None:
            try:
                font = ImageFont.truetype("C:/Windows/Fonts/arial.ttf", size)
            except: pass

        # 5. Cálculo de dimensiones para centrado
        bbox = draw.textbbox((0, 0), text, font=font)
        text_w = bbox[2] - bbox[0]
        
        x = xy[0]
        y = xy[1]
        
        # 6. Si x es -1, se centra el texto horizontalmente en la imagen
        if x == -1: 
            W = frame.shape[1]
            x = (W - text_w) // 2

        draw.text((x, y), text, font=font, fill=color_rgb)
        
        # Retorna la imagen convertida de nuevo a formato OpenCV
        return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

    def detectar_gesto_actual(self, frame):
        """
        Procesa el frame para detectar manos o caras.
        Prioriza la detección de manos. Si la confianza es baja (<0.8),
        intenta detectar gestos faciales.
        """
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gesto = "0_Neutro"
        max_prob = 0.0

        # Detección de Manos 
        res_hand = self.hands.process(rgb)
        if res_hand.multi_hand_landmarks:
            lm = res_hand.multi_hand_landmarks[0]
            self.mp_draw.draw_landmarks(frame, lm, self.mp_hands.HAND_CONNECTIONS)
            features = get_hand_features(lm)
            try:
                pred = self.model_hand.predict([features])[0]
                prob = np.max(self.model_hand.predict_proba([features]))
                if prob > 0.5:
                    label = self.map_hand[pred]
                    if "Neutro" not in label:
                        gesto = label
                        max_prob = prob
            except: pass

        # Detección Facial
        if max_prob < 0.8:
            res_face = self.face_mesh.process(rgb)
            if res_face.multi_face_landmarks:
                lm = res_face.multi_face_landmarks[0]
                features = get_face_features(lm)
                try:
                    pred = self.model_face.predict([features])[0]
                    prob = np.max(self.model_face.predict_proba([features]))
                    if prob > 0.6:
                        label = self.map_face[pred]
                        if "Neutro" not in label and prob > max_prob:
                            gesto = label
                            max_prob = prob
                except: pass
        
        return gesto, max_prob

    def draw_ui(self, frame, main, sub="", color=(255, 255, 255)):
        """Dibuja la interfaz de usuario estándar (barra superior) usando OpenCV."""
        H, W, _ = frame.shape
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (W, 110), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
        cv2.putText(frame, main, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 3)
        cv2.putText(frame, sub, (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2)
        cv2.putText(frame, f"Score: {self.score}", (W-180, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, COLORS['neon_yellow'], 2)

    def add_sequence_step(self):
        """
        Genera un nuevo paso en la secuencia del juego.
        Implementa la lógica de dificultad progresiva basada en la puntuación:
        - Nivel 1: Simón vs Modesto.
        - Nivel 2 (>30 pts): Trampas semánticas (Simón sin 'dice').
        - Nivel 3 (>60 pts): Trampas de sujeto (Falta 'Simón').
        """
        accion = random.choice(self.acciones_disponibles)
        
        opciones = ['valid', 'modesto']
        if self.score >= 30: opciones.append('simon_fake')
        if self.score >= 60: opciones.append('no_name')

        tipo = random.choice(opciones)
        
        # Configuración del encabezado y flag de trampa según el tipo generado
        if tipo == 'valid':
            header = "Simón dice:"
            es_trampa = False
        elif tipo == 'modesto':
            header = "Modesto dice:"
            es_trampa = True
        elif tipo == 'simon_fake':
            header = "Simón:"   # Falta el verbo "dice"
            es_trampa = True
        elif tipo == 'no_name':
            header = "Dice:"    # Falta el sujeto "Simón"
            es_trampa = True
            
        self.sequence.append({
            'action': accion,
            'header': header,
            'is_trap': es_trampa,
            'type': tipo
        })

    def dibujar_tarjeta_hud(self, frame, titulo, linea1, linea2, progreso_tiempo):
        """
        Dibuja una tarjeta semitransparente respetando la lógica de 1 o 2 líneas de texto.
        """
        H, W, _ = frame.shape
        
        # Dimensiones de la tarjeta
        card_w, card_h = 600, 250
        x1 = (W - card_w) // 2
        y1 = (H - card_h) // 2
        x2, y2 = x1 + card_w, y1 + card_h
        
        # 1. Fondo y Transparencia 
        overlay = frame.copy()
        cv2.rectangle(overlay, (x1, y1), (x2, y2), (20, 20, 20), -1)
        alpha = 0.7
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
        
        # 2. Barra de Tiempo         
        bar_height = 10
        bar_width = int(card_w * progreso_tiempo)
        bar_color = (0, 255, 0) if progreso_tiempo > 0.3 else (0, 0, 255)
        cv2.rectangle(frame, (x1, y2 - bar_height), (x1 + bar_width, y2), bar_color, -1)
        
        # 3. Texto
        velocidad_cambio = 3.0
        idx_color = int(time.time() * velocidad_cambio) % len(COLORES_ANIMACION)
        color_dinamico = COLORES_ANIMACION[idx_color]
        
        frame = self.poner_texto_utf8(frame, titulo, (-1, y1 + 25), 
                                      color=color_dinamico, size=40, font_path=FONT_CUSTOM)
        
        # 4. Texto (Instrucción)
        area_texto_y = y1 + 80
        
        if linea2: # Caso: DOS LÍNEAS
            frame = self.poner_texto_utf8(frame, linea1, (-1, area_texto_y), 
                                          color=COLORS['white'], size=50, font_path=FONT_CUSTOM)
            frame = self.poner_texto_utf8(frame, linea2, (-1, area_texto_y + 60), 
                                          color=COLORS['white'], size=50, font_path=FONT_CUSTOM)
        else: # Caso: UNA LÍNEA (Centrada)
            frame = self.poner_texto_utf8(frame, linea1, (-1, area_texto_y + 30), 
                                          color=COLORS['white'], size=60, font_path=FONT_CUSTOM)
        
        return frame

    def run(self):
        """
        Bucle principal de ejecución.
        Gestiona la captura de video, la máquina de estados y la lógica de renderizado.
        """        
        # Configurar ventana para pantalla completa
        window_name = 'SIMON MEMORY GAME'
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

        try:
            while self.cap.isOpened():
                ret, frame = self.cap.read()
                if not ret: break
                frame = cv2.flip(frame, 1) # Efecto espejo
                H, W, _ = frame.shape
                
                # Detección y estabilización de gestos 
                raw_gesto, prob = self.detectar_gesto_actual(frame)
                
                if raw_gesto == self.last_gesture:
                    self.consecutive_frames += 1
                else:
                    self.last_gesture = raw_gesto
                    self.consecutive_frames = 0
                
                # Se requiere consistencia en varios frames para evitar parpadeos
                gesto_estable = raw_gesto if self.consecutive_frames > 2 else "..."

                texto_detectado = DETECCION.get(gesto_estable, gesto_estable)

                # Debug en pantalla (esquina inferior)
                frame = self.poner_texto_utf8(frame, f"Detectado: {texto_detectado}", (20, H-30), 
                            color=COLORS['neon_green'], size=20, font_path=FONT_CUSTOM)

                # MÁQUINA DE ESTADOS DEL JUEGO 
                if self.state == "MENU":
                    cv2.rectangle(frame, (0,0), (W,H), (0,0,0), -1) 
                    frame = self.poner_texto_utf8(frame, "SIMON SAYS", (-1, H//2 - 100), COLORS['neon_purple'], 90, FONT_CUSTOM)
                    frame = self.poner_texto_utf8(frame, "[ ESPACIO para empezar ]", (-1, H - 150), COLORS['white'], 30)

                    if cv2.waitKey(1) & 0xFF == 32: # Tecla Espacio
                        self.sequence = []
                        self.score = 0
                        self.add_sequence_step()
                        self.state = "SHOW_NEW_STEP"
                        self.current_step_idx = 0
                        self.speech_triggered = False
                        self.timer_start = time.time()

                elif self.state == "SHOW_NEW_STEP":
                    last_step = self.sequence[-1]
                    elapsed = time.time() - self.timer_start
                    
                    # Cálculo de barra de tiempo
                    tiempo_restante_pct = 1.0 - (elapsed / self.tiempo_memorizar)
                    if tiempo_restante_pct < 0: tiempo_restante_pct = 0

                    texto_accion = INSTRUCCIONES.get(last_step['action'], last_step['action'])
                    header_text = last_step['header']

                    # Separar texto en dos líneas 
                    MAX_CHARS = 18 
                    palabras = texto_accion.split()
                    linea1 = ""
                    linea2 = ""

                    for palabra in palabras:
                        if len(linea1 + " " + palabra) <= MAX_CHARS:
                            linea1 += (" " if linea1 else "") + palabra
                        else:
                            linea2 += (" " if linea2 else "") + palabra

                    # Voz
                    if not self.speech_triggered:
                        frase = f"{header_text} {texto_accion}"
                        self.hablar(frase)
                        self.speech_triggered = True

                    # UI:
                    frame = self.dibujar_tarjeta_hud(
                        frame, 
                        header_text, 
                        linea1,   
                        linea2,
                        tiempo_restante_pct
                    )

                    if elapsed > self.tiempo_memorizar: 
                        self.state = "WAIT_NEUTRAL" 
                        self.current_step_idx = 0
                        self.waiting_neutral = True

                elif self.state == "WAIT_NEUTRAL":
                    # Espera a que el jugador vuelva a posición neutra antes del turno
                    self.draw_ui(frame, "VUELVE A NEUTRO", "Preparando siguiente...", color=COLORS['ligth_grey'])
                    if gesto_estable == "0_Neutro":
                        self.state = "PLAYER_TURN"
                        self.timer_start = time.time()
                        self.neutral_start_time = None

                elif self.state == "PLAYER_TURN":
                    # Turno del jugador: Debe repetir la secuencia paso a paso
                    step = self.sequence[self.current_step_idx]
                    elapsed = time.time() - self.timer_start
                    remaining = self.turn_time - elapsed
                    
                    idx_show = self.current_step_idx + 1
                    total = len(self.sequence)
                    
                    instr = f"Repite el paso {idx_show}/{total}"
                    self.draw_ui(frame, instr, f"Tiempo: {remaining:.1f}s")
                    
                    round_passed = False
                    round_failed = False
                    
                
                # --- Lógica de Detección de Neutro Continuo (Solo para trampas) ---
                    if step['is_trap']:
                        # === Lógica Trampas ===
                        # Si es trampa: DEBE mantenerse neutro
                        # Si hace CUALQUIER OTRA COSA -> FALLA
                        if gesto_estable == "0_Neutro":
                            if self.neutral_start_time is None:
                                self.neutral_start_time = time.time()
                            
                            tiempo_quieto = time.time() - self.neutral_start_time
                            
                            # Si aguanta quieto el tiempo configurado, gana
                            if tiempo_quieto >= self.tiempo_espera_trampa:
                                round_passed = True
                        elif gesto_estable != "...": 
                            # Si NO es neutro y NO es indefinido (...) => Ha hecho un gesto => FALLO
                            round_failed = True
                            if step.get('type') == 'modesto': self.fail_reason = "¡Era Modesto! No debías moverte."
                            elif step.get('type') == 'simon_fake': self.fail_reason = "¡Dijo 'Simon' pero no 'Dice'!"
                            elif step.get('type') == 'no_name': self.fail_reason = "¡Nadie dijo 'Simon'!"
                            else: self.fail_reason = "¡Era una trampa! Debías quedarte neutro."
                        
                        # Si sobrevive el tiempo del turno sin fallar, gana.
                        elif remaining <= 0:
                            round_passed = True

                    else:
                        # === Lógica Normal (Simón Dice) ===
                        if gesto_estable == step['action']:
                            round_passed = True
                        
                        # Si hace un gesto que NO es el correcto y NO es neutro -> FALLO
                        elif gesto_estable != "0_Neutro" and gesto_estable != "...":
                            round_failed = True
                            accion_erronea = DETECCION.get(gesto_estable, gesto_estable)
                            accion_esperada_error = ERROR.get(step['action'], step['action'])
                            self.fail_reason = (
                                f"{f'¡Mal gesto! Hiciste `{accion_erronea}´'.center(80)}\n"
                                f"{f'cuando debías {accion_esperada_error}'.center(80)}"
                            )

                        elif remaining <= 0:
                            round_failed = True
                            accion_esperada_error = ERROR.get(step['action'], step['action'])
                            self.fail_reason = f"¡Lento! Tenias que {accion_esperada_error}"

                    # --- Transiciones de Estado ---
                    if round_failed:
                        self.play_sound("fail")
                        self.state = "GAMEOVER"
                    elif round_passed:
                        self.play_sound("success") # Feedback sonoro
                        cv2.circle(frame, (W-50, H-50), 40, COLORS['neon_green'], -1) # Feedback visual verde
                        self.score += 10
                        self.current_step_idx += 1
                        
                        # Verifica si quedan pasos en la secuencia actual
                        if self.current_step_idx < len(self.sequence):
                            self.state = "WAIT_NEUTRAL"
                        else:
                            self.state = "SUCCESS_SEQUENCE"
                            self.timer_start = time.time()

                elif self.state == "SUCCESS_SEQUENCE":
                    # Ronda completada con éxito
                    overlay = frame.copy()
                    cv2.rectangle(overlay, (0, 0), (W, H), (0, 50, 0), -1)
                    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)

                    frame = self.poner_texto_utf8(frame, "¡RONDA COMPLETADA!", (W//2 - 180, H//2 - 50), 
                                                  color=COLORS['white'], size=50, font_path=FONT_CUSTOM)
                    
                    if time.time() - self.timer_start > self.tiempo_entre_rondas:
                        self.add_sequence_step()    
                        self.state = "SHOW_NEW_STEP" 
                        self.speech_triggered = False
                        self.timer_start = time.time()

                elif self.state == "GAMEOVER":
                    # Pantalla de fin de juego con razón del fallo
                    overlay = frame.copy()
                    cv2.rectangle(overlay, (0, 0), (W, H), (0, 0, 50), -1)
                    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
                    
                    cv2.putText(frame, "GAME OVER", (W//2 - 180, H//2 - 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)
                    
                    # Mensaje de error personalizado usando la fuente custom
                    frame = self.poner_texto_utf8(frame, self.fail_reason, (-1, H//2 + 20), 
                                                  color=COLORS['ligth_grey'], size=30, font_path=FONT_CUSTOM)

                    cv2.putText(frame, "R: Reiniciar   |   ESC: Salir", (W//2 - 200, H - 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)    
                    
                    if cv2.waitKey(1) & 0xFF == ord('r'):
                        self.score = 0
                        self.sequence = []
                        self.state = "MENU"

                cv2.imshow('SIMON MEMORY GAME', frame)
                if cv2.waitKey(1) & 0xFF == 27: break # Tecla ESC para salir
        
        finally:
            self.cap.release()
            cv2.destroyAllWindows()
            pygame.quit()
        


### Iniciar Juego

In [272]:
if __name__ == "__main__":
    try:
        game = SimonGame()
        game.run()
    except Exception as e:
        print(f"Ocurrió un error: {e}")
    finally:
        cv2.destroyAllWindows()

--- SIMON SAYS: MEMORY ---


c:\Users\laura\anaconda3\envs\VC_final\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\laura\anaconda3\envs\VC_final\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\laura\anaconda3\envs\VC_final\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\User